In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from openai import OpenAI

"""header={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)\
            Chrome/133.0.0.0 Safari/537.36"}
            """
header={"Content-Type":"Application/json"}

class Website:
    def __init__(self,url):
        self.url=url
        #retrieve the website using the html with header info required on title,text elimination the irrelevant
        response=requests.get(url,headers=header)
        print(f"Status Code: {response.status_code}")  # Should be 200
        print(f"Page Content:\n{response.text[:500]}")  # Show first 500 chars
        
        soup=BeautifulSoup(response.content,'html.parser')
                
        self.title=soup.title.string if soup.title else "No title of the website"
        for unnecessary in soup.body(["script","style","img"]):
            unnecessary.decompose()
            
        self.text=soup.body.get_text(separator="\n" ,strip=True) if soup.body else "No body-text of website"
        
#creating an object for the website
code=Website("https://edwarddonner.com")


In [ ]:
system_prompt="i need your help in categorising the website of the industrial sector basing on its sectors and all its involved in "

def user_prompt_ed(code):
    user_prompt=f"The generated title is {code.title}"
    user_prompt +="retrieve all the necessary information creating a summary of the important notes \
                with all the necessary contact information of each summarised sector \
                an easy means of how to make a events in calendar \n"
    user_prompt +=code.text
    return user_prompt

In [ ]:
def messages_ed(code):
    messages=[
        {"role":"user" ,"content":user_prompt_ed(code)},
        {"role":"system" ,"content":system_prompt}
    ]
    return messages
    

In [ ]:
payload={
    "model":"llama3.2",
    "messages":messages_ed(code),
    "stream":False
}

In [ ]:
"""
def bronchure(code):
    ollama_via_openai=OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")
    response=ollama_via_openai.chat.completions.create(model="llama3.2" ,messages=messages_ed(code))
    return response.choices[0].message.content   
print(response
"""

In [ ]:
import ollama

response=ollama.chat(model="llama3.2" , messages=messages_ed(code) )
print(response['message']['content'])


In [ ]:
def bronchure(code):
    try:
        ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
        response = ollama_via_openai.chat.completions.create(model="llama3.2", messages=messages_ed(code))
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

code_ed = bronchure(code)
display(Markdown(code_ed))

In [ ]:
MODEL_API="http://localhost:11434/api/chat" 
HEADER={"Content-Type":"application/json"}
response=requests.post(MODEL_API, json=payload ,headers=header)
code_ed=response.json()['message']['content']
print(display(Markdown(code_ed)))

In [ ]:
system_prompt = "I need your help in categorising the website of the industrial sector basing on its sectors and all its involved in"

def user_prompt_ed(code):
    user_prompt = f"The generated title is {code.title}\n"
    user_prompt += "retrieve all the necessary information creating a summary of the important notes \\\n"
    user_prompt += "with all the necessary contact information of each summarised sector \\\n"
    user_prompt += "an easy means of how to make a events in calendar \n"
    user_prompt += code.text
    return user_prompt

def messages_ed(code):
    messages = [
        {"role": "user", "content": user_prompt_ed(code)},
        {"role": "system", "content": system_prompt}
    ]
    return messages

def bronchure(code):
    try:
        ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
        response = ollama_via_openai.chat.completions.create(model="llama3.2", messages=messages_ed(code))
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

code_ed = bronchure(code)
display(Markdown(code_ed))